![AuroraAI](images/auroraai-small.png)

# Data preparation script for AuroraAI kouluterveyskysely 2021 data

This script prepares the 2021 kouluterveyskysely data for clustercards generation.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime
from collections import defaultdict
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from yamlconfig import read_config

## Read and define variables

In [ ]:
config = read_config()
c = config['ktk21']
print('Settings:')
print(c)

In [ ]:
DATADIR = c['datadir']
DATAFILE = c['datafile']
METAFILE = c['metafile']
OUTPREFIX = c['outprefix']
FILTER_YEAR = c['filter_year']
FILTER_MUNICIPALITY = c['filter_municipality']

INDEX_COL = None
CSV_SEP = ';'
NAN_LIMIT = 0.25

In [ ]:
datafilename = "{}/{}".format(DATADIR, DATAFILE)
metafilename = "{}/{}".format(DATADIR, METAFILE)
assert os.path.isfile(datafilename), "File missing"
assert os.path.isfile(metafilename), "File missing"

## Read data

### Read metadata for variables

In [ ]:
df_labels = pd.read_excel(metafilename, index_col="Muuttuja")
df_labels.replace(np.nan, '0', inplace=True)
for c in df_labels.columns:
    if c in ["Lyhyt kuvaus", "Kuvaus", "Kommentit"]: continue
    df_labels[c] = df_labels[c].astype(int)
df_labels.info()
df_labels.sample(5)

#### Background variables

The following variables will not be included in the clustering or any other analysis:

In [ ]:
background = df_labels['Taustamuuttuja']>0
background = list(background[background].index.values)
print(background, len(background))

#### Variables to be inverted

The values of the following variables will be inverted:

In [ ]:
invert = []
if 'Kääntö' in df_labels:
    invert =  df_labels['Kääntö']>0
    invert = list(invert[invert].index.values)    
print(invert)

### Read actual data from CSV

In [ ]:
df = pd.read_csv(datafilename, sep=CSV_SEP, index_col=INDEX_COL, dtype=float, na_values=" ")
df.info()
df.sample(5)

In [ ]:
df.describe()

In [ ]:
len(df.columns)

In [ ]:
def plot_hist(var):
    counts = df[var].value_counts(dropna=False)
    counts.plot.bar(title=var, grid=True);

In [ ]:
plot_hist('vuosi')

In [ ]:
plot_hist('kunta_koodi_tku_jkl')

In [ ]:
plot_hist('ind_familyeconomic')

In [ ]:
plot_hist('sukupuoli')

In [ ]:
df['sukupuoli'].value_counts()

## Process data

### NaNs

In [ ]:
ser_nans = df.isnull().sum()
ser_too_many_nans = ser_nans[ser_nans>NAN_LIMIT*len(df)]
too_many_nans = list(ser_too_many_nans.index.values)
print('Removing variables with more than {} NaNs:'.format(NAN_LIMIT*len(df)))
print(ser_too_many_nans)
print()
df = df.drop(ser_too_many_nans.index, axis=1)
df.info(verbose=False)

In [ ]:
df.isnull().sum().sort_values(ascending=False).plot()

In [ ]:
if 1: 
    df_mode = df.mode().transpose().squeeze()
    df_mean = df.mean()

    for c in ['luokka', 'sukupuoli']: 
        df[c].fillna(df_mode[c], inplace=True)

    for c in df.columns: 
        df[c].fillna(df_mean[c], inplace=True)

In [ ]:
if 0:
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn.impute import KNNImputer

    scaler = StandardScaler()
    df_sc = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

    imputer = KNNImputer(n_neighbors=3)
    df_imp = pd.DataFrame(imputer.fit_transform(df_sc),columns = df.columns)

In [ ]:
df

In [ ]:
assert not df.isnull().sum().sum(), "NaNs exist"

### Invert variables

In [ ]:
for iv in invert:
    print('Inverting variable:', iv)
    df[iv] = df[iv].max() - df[iv] 

### Filter rows

In [ ]:
if FILTER_YEAR is not None:
    print('Filtering data with FILTER_YEAR={}'.format(FILTER_YEAR))
    vuosi_int = df['vuosi'].astype(int)
    assert FILTER_YEAR in vuosi_int.values, "FILTER_YEAR={} not found".format(FILTER_YEAR)
    df = df[vuosi_int == FILTER_YEAR]
    df.info()
    display(df.sample(5))

In [ ]:
if FILTER_MUNICIPALITY is not None:
    print('Filtering data with FILTER_MUNICIPALITY={}'.format(FILTER_MUNICIPALITY))
    kuntakoodi_int = df['kunta_koodi_tku_jkl'].astype(int)
    assert FILTER_MUNICIPALITY in kuntakoodi_int.values, "FILTER_MUNICIPALITY={} not found".format(FILTER_MUNICIPALITY)
    df = df[kuntakoodi_int == FILTER_MUNICIPALITY]
    df.info()
    df.sample(5)

### Background variables

In [ ]:
for v in too_many_nans:
    if v in background:
        print('Removing', v, 'from background variables')
        background.remove(v)
print(background)

In [ ]:
df_bg = df[background]
df = df.drop(background, axis=1)
print(len(background), 'background variables removed:', background)
print('Actual data has', len(df), 'rows,', len(df.columns), 'columns')
print('Background data has', len(df_bg), 'rows,', len(df_bg.columns), 'columns')

## Save data as a CSV file

In [ ]:
now = datetime.now()
todaystr = now.strftime("%Y-%m-%d")
outfile = "{}/{}-data-{}.csv".format(DATADIR, OUTPREFIX, todaystr)
print(outfile)
df.to_csv(outfile, index=False)

In [ ]:
now = datetime.now()
todaystr = now.strftime("%Y-%m-%d")
outfile = "{}/{}-bg-{}.csv".format(DATADIR, OUTPREFIX, todaystr)
print(outfile)
df_bg.to_csv(outfile, index=False)